# Instantiation

In [ ]:
import siibra
from packaging.version import Version
assert Version(siibra.__version__) >= Version('1.0a08')

import matplotlib
from nilearn import plotting

# ignore the following lines at this point - we just touch some objects to trigger data loading 
# while we still take the introduction


For the purpose of this tutorial, we request some objects here already to retrieve data while your tutor still does the introductory remarks :-) Please just run the cell and ignore for now. 

In [ ]:
julich_brain = siibra.parcellations.get('julich 3.1')
julich_pmaps = julich_brain.get_map('mni152', 'statistical')
julich_pmaps.fetch(region='4a left')
siibra.warm_cache()

# Part I: Accessing reference spaces, parcellations and regions


## Instance tables of key concepts

`siibra` is structured around the key concepts atlas, reference space, parcellation and parcellation region. Each of these concepts has a specific type. `siibra` comes with preconfigured instances of these concepts, which can be easily accessed via *instance tables*. When you load siibra for the first time, it pulls this preconfiguration information from our online repository and caches it on your computer. Therefore, `siibra` will be a bit slower when you use it for the first time (or after a version upgrade).

Here is an overview of the key concepts:

| Concept | siibra type | instance table | description |
| :-- | :-- | :-- | :-- |
| Atlases | Atlas | `siibra.atlases` | A collection of related reference spaces and parcellations, typically per species |
| Reference spaces | Space | `siibra.spaces` | 3D coordinate systems of the brain |
| Parcellations | Parcellation | `siibra.parcellations` | Different brain parcellations schemes with their region hierarchies |
| regions | Region | - | Structures defined within a parcellation, each representing a subtree of a parcellation's hierarchy| 

**Note:** These concepts are just semantic objects - they mostly give names and relationships to atlas concepts. We will deal with parcellation maps in the next notebook

In [ ]:
import siibra

## Select parcellations from their instance table

The instance table for parcellations is `siibra.parcellations`. To get an overview, we can simply print its elements.

In [ ]:
siibra.parcellations

In [ ]:
print(siibra.parcellations)

To actually access an element from an instance table, there are several options:

 - by tab-completion on an interactive Python shell. This is very convenient, since it allows you to browse while typing.
 - by fuzzy keyword matching via the get() function
 - By using the index operator `[]` with keywords or numbers

In [ ]:
siibra.parcellations.JULICH_BRAIN_CYTOARCHITECTONIC_ATLAS_V3_0_3

In [ ]:
siibra.parcellations.get('bundles')

In [ ]:
# this is equivalent to the above
siibra.parcellations['bundles']

## Your turn! Browse predefined reference spaces

<div class="alert alert-block alert-info">
Access the MNI 152 ICBM 2009c nonl asymmetric space.
</div>

## Properties of parcellation objects

The parcellations contain some metadata, such as name, description and related publication:

In [ ]:
julich_brain = siibra.parcellations.get('julich')
print(julich_brain.name)

In [ ]:
print(julich_brain.description)

for p in julich_brain.publications:
    print("\n" + p['citation'])

## Brain regions

Providing all brain regions through instance tables would not be helpful, since there are thousands of them. Regions are organized into hierarchical trees attached to a parcellation.  A parcellation object in siibra is actually a special type of region, namely the top node of a region tree with some additional metadata. 

Other regions may be the root of a more fine-grained subtree, or leafs without children.

We can print the subtree of each region object using the `Region.tree2str` function.

In [ ]:
print(julich_brain.tree2str())

To access individual brain regions, we typically pass over a unique part of their name to the parcellation (or any parent region):

In [ ]:
amygdala = julich_brain.get_region('amygdala')
print(amygdala.tree2str())

# Part II - Fetching reference templates and parcellation maps

## Volumetric maps

The same parcellation can be mapped in different refernce spaces. So to request a parcellation map, we have to link a parcellation with a reference space. In siibra, we request the map from the parcellation.

In [ ]:
julich_brain = (
    siibra.parcellations.JULICH_BRAIN_CYTOARCHITECTONIC_ATLAS_V3_0_3
)
julich_mpm = julich_brain.get_map(
    space=siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC
)

At this point, we just have a parcellation map object. The image volume has not yet been loaded. To do so, we use the `fetch()` method.

In [ ]:
mpm_img = julich_mpm.fetch()
mpm_img

As you see, the image data is provided as a Nifti1Image, which is very common to use in neuroscience. siibra represents most image data in this format. We can use common libraries to visualize the image - we recommend the excellent `nilearn.plotting` tools. Some parcellation maps in siibra provide their own colormap that can be used.

In [ ]:
plot_args = {"symmetric_cmap": False, "colorbar": False}
plotting.plot_roi(
    mpm_img,
    title=julich_brain.name,
    cmap=julich_mpm.get_colormap(),
)

We can also fetch the map of a single region.

In [ ]:
motor_map = julich_mpm.fetch(region='4a left')
plotting.plot_roi(motor_map, title='4a left')

## Regional probability maps

So far we looked at labelled parcellation maps. To access probability maps, we explicitly request the "statistical" maptype. Everything else works the same way.

In [ ]:
julich_pmaps = julich_brain.get_map(
    siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC,
    maptype=siibra.MapType.STATISTICAL
)
motor_pmap = julich_pmaps.fetch(region='4a left')
plotting.plot_stat_map(motor_pmap, title='4a left')

## Surface maps

Julich-Brain is also provided as a surface map in fsaverage. fsaverage represents a different space. This time, instead of a NIfTI image, we obtain a mesh structure, which is a dictionary with vertices, faces and vertex labels.

In [ ]:
julich_surfmap = julich_brain.get_map(siibra.spaces.FREESURFER_FSAVERAGE)
surf = julich_surfmap.fetch()
surf

In [ ]:
plotting.view_surf(
    surf_mesh=[surf['verts'], surf['faces']],
    surf_map=surf['labels'], 
    cmap=julich_surfmap.get_colormap(),
    **plot_args
)

### Your turn!

<div class="alert alert-block alert-info">
Load and display the DiFuMo 64 map in MNI152 space.
</div>

## Reference templates

Just as we can fetch maps, we can fetch the reference template of a space. Let's do this to fetch the BigBrain model!

Per default, fetch() will download a low-resolution version. We will talk about higher resolutions later on.

In [ ]:
julich_brain.shortname

In [ ]:
bigbrain_space = siibra.spaces.BIGBRAIN_MICROSCOPIC_TEMPLATE_HISTOLOGY
bigbrain_template = bigbrain_space.get_template()
plotting.plot_img(bigbrain_template.fetch(), bg_img=None, cmap='gray')

# Part III - Assigning locations to brain structures

## Specifying brain locations

Locations in the brain are often specified by coordinates, or peaks/blobs in images. siibra has predefined location types for points, sets of points, and bounding boxes in reference spaces.

siibra tries to ensure that location objects, just as maps and images, are clearly linked to a reference space.

In [ ]:
# create a point by specifying coordinates. You can do this in the viewer!
point = siibra.locations.Point("24.150mm, -18.000mm, 42.150mm", space='mni152')

In [ ]:
plotting.plot_img(mpm_img, cut_coords=tuple(point), cmap=julich_mpm.get_colormap())

In [ ]:
point = siibra.from_json("""
{
  "@id": "676457d9",
  "@type": "https://openminds.ebrains.eu/sands/CoordinatePoint",
  "coordinateSpace": {
    "@id": "minds/core/referencespace/v1.0.0/a1655b99-82f1-420f-a3c2-fe80fd4c8588"
  },
  "coordinates": [
    {
      "@id": "9837b595",
      "@type": "https://openminds.ebrains.eu/core/QuantitativeValue",
      "value": 13.0557,
      "unit": {
        "@id": "id.link/mm"
      }
    },
    {
      "@id": "d3382985",
      "@type": "https://openminds.ebrains.eu/core/QuantitativeValue",
      "value": 0.485,
      "unit": {
        "@id": "id.link/mm"
      }
    },
    {
      "@id": "84fc5d5d",
      "@type": "https://openminds.ebrains.eu/core/QuantitativeValue",
      "value": 40.4396,
      "unit": {
        "@id": "id.link/mm"
      }
    }
  ]
}
""")

In [ ]:
view = plotting.plot_img(bigbrain_template.fetch(), cut_coords = tuple(point), cmap='gray')
view.add_markers([tuple(point)])

## Probabilistic assignment of points

Parcellation maps in siibra can assign their structures to locations and images.

Note that siibra can warp locations between different template spaces. It does this automatically in many cases. For example, the Julich-Brain probability maps are defined in MNI space, while our point above is from BigBrain. siibra will detect this and convert accordingly for the assignment.

In [ ]:
julich_pmaps.assign(point)

## Your turn!

<div class='alert alert-block alert-info'>
Pick a location from the online viewer, generate a siibra.Point and assign it to brain structures.
</div>

## Probabilistic assignment of image signals

Just like points, siibra can assign brain regions to structures in an image volume. Here we load an example volume, and feed it to the same assign() method as the point. Since we have here image information, the asignment produces some scores for each match. siibra will split the volume into disconnected components. This typically results in many more assignments, so we will filter the resulting list using the scores.

In [ ]:
img = siibra.volumes.from_file(
    'ohbm-2023-example-input.nii.gz',
    space='mni152',
    name='Example input'
)
plotting.plot_stat_map(img.fetch())

In [ ]:
assignments = julich_pmaps.assign(img)
assignments[assignments.correlation > 0.3].sort_values("input containedness")  # refer to pandas documentation for more...

# Part IV - Collecting multimodal features


`siibra` provides access to data features of different modalities. The features and their query functions are bundled in the module `siibra.features`. We can choose different types of features from this module. The feature types are organized in a hierarchy under the most abstract type `siibra.features.Feature`. All other feature types are subclasses of it. Let's look at this hierarchy.

In [ ]:
siibra.features.render_ascii_tree("Feature")

### Densities of neurotransmitter receptors

Features can be queried for brain regions, parcellations and location objects (such as volumes of interest in a refrence space) with `siibra.features.get()`, which accepts a query object and a feature type. It will query all subclasses of the given feature type, if any. Here is a simple example for getting a receptor density fingerprint in region V1. The data is of tabular type, provided as a pandas dataframe.

In [ ]:
v1 = siibra.get_region(parcellation='julich 2.9', region='v1 left')

In [ ]:
features = siibra.features.get(
    v1, siibra.features.molecular.ReceptorDensityFingerprint
)

# fetch the first one
f = features[0]
print(f.name)
f.data.T  # we transpose the table for display

In [ ]:
# besides the data table, the features have some additional metadata
print(f.modality)
print(f.description)

In [ ]:
f.plot()

### Understanding what was matched

In [ ]:
f.last_match_description

In [ ]:
print(julich_brain.get_region("occipital cortex").tree2str())

In [ ]:
features = siibra.features.get(
    julich_brain.get_region("occipital cortex"),
    siibra.features.molecular.ReceptorDensityFingerprint
)

In [ ]:
for f in features:
    print(f.last_match_description)

## Your turn! 

<div class="alert alert-block alert-info">
    Query layerwise cell densities for V1
</div>

### Gene Expressions from the Allen Atlas

siibra also implements a live queryto gene expression data from the Allen atlas to extract regional gene expression levels. Gene expressions are linked to atlas regions by coordinates of their probes in MNI space. When called with a brain region, siibra.features.get will generate a mask of this region in MNI space to filter the probes. It provides the regional expression levels as tabular data, together with the MNI coordinates.

In [ ]:
features = siibra.features.get(
    v1, siibra.features.molecular.GeneExpressions, gene=["TAC1", "MAOA", "GABARAPL2"]
)
fig = features[0].plot()
features[0].data

In [ ]:
# We can plot the MNI coordinates to localize the measures and verify they are located in V1.
locations = siibra.PointSet(features[0].data.mni_xyz.tolist(), space="mni152")

from nilearn import plotting
mask = v1.get_regional_map("mni152")
display = plotting.plot_glass_brain(mask.fetch(), cmap='viridis')
display.add_markers(locations.as_list(), marker_size=2) 

### Structural and functional connectivity

`siibra` provides connectivity matrices with parcellation averaged structural and functional measurments for different subjects of different cohorts. Here we request some streamline counts for Julich Brain.

In [ ]:
features = siibra.features.get(
    siibra.parcellations.get('julich 2.9'),
    siibra.features.connectivity.StreamlineCounts
)
f = features[0]

Let's check the cohort and subject ids of the first connectivity feature.

In [ ]:
print(f.cohort)
print(f.indices)

We can retrieve the matrix of a single subject using `get_matrix()`. If we leave the subject specification out, `siibra` will compute the mean matrix across subjects.

Again, the result is a pandas dataframe, with the notable property that the row and column indices are full region objects for further reference. This implies in particular, that we can directly associate each measure with the corresponding information in the parcellation, and with a mask of a parcellation map.

In [ ]:
sc_hcp_000 = f.get_element('000')
sc_hcp_000

In [ ]:
# we can also plot the matrix
sc_hcp_000.plot(logscale=True)

As an example, we retrieve the centroids in MNI152 space and plot the connnectivity graph in 3D:

In [ ]:
node_coords = sc_hcp_000.compute_centroids(space='mni152')
plotting.view_connectome(
    adjacency_matrix=sc_hcp_000.data,
    node_coords=node_coords,
    edge_threshold="99%",
    node_size=3, colorbar=False,
    edge_cmap="bwr"
)

## Working with BigBrain data

### Extracting chunks from the 20 micron model

To access full resolution data of BigBrain, we specify a bounding box in the physical space. For now, we just define a volume of interest from two corner points in the histological space. We specify the points with a string representation, which could be conveniently copy pasted from the interactive viewer siibra explorer. Note that the coordinates can be specified by 3-tuples, and in other ways.

In [ ]:
voi = siibra.locations.BoundingBox(
    point1="-30.590mm, 3.270mm, 47.814mm",
    point2="-26.557mm, 6.277mm, 50.631mm",
    space='bigbrain'
)

# Note: we can reuse the template object from above,
# just fetch with different parameters
bigbrain_chunk = bigbrain_template.fetch(voi=voi, resolution_mm=0.02)
plotting.plot_img(bigbrain_chunk, cmap='gray')

### Loading cortical profiles (by Wagstyl et al.)

Cortical staining profiles in BigBrain have been precomputed by Konrad Wagstyl and colleagues. Siibra has integrated those as a feature type, for the left hemishere. We can thus query profiles by region!

In [ ]:
features = siibra.features.get(
    siibra.get_region('julich', '4p left'),
    siibra.features.cellular.BigBrainIntensityProfile
)
print(len(features))
f = features[0]

In [ ]:
f.data

In [ ]:
f.plot()

### 1 micron sections

HIBALL has released a range of 1 micron scans of BigBrain sections across the brain. Siibra can find those as VolumeOfInterest features. The result is a high-resolution image structure, just like the bigbrain template.

In [ ]:
hoc5l = siibra.get_region('julich', 'hoc5 left')
features = siibra.features.get(
    hoc5l,
    siibra.features.cellular.CellbodyStainedSection
)

In [ ]:
# let's see the names of the found features
for f in features:
    print(f.name)

In [ ]:
# let's fetch the 1 micron section at a lower resolutioin, and display in 3D space.
section1402 = features[3]
plotting.plot_img(
    section1402.fetch(),
    bg_img=bigbrain_template.fetch(),
    title="#1402",
    cmap='gray'
)

In [ ]:
# Let's fetch a crop inside hoc5 at full resolution.
# we intersect the bounding box of hoc5l and the section
hoc5_bbox = hoc5l.get_bounding_box('bigbrain').intersection(section1402.boundingbox)
print(f"Size of the bounding box: {hoc5_bbox.shape}")

# this is quite large, so we shrink it
voi = hoc5_bbox.zoom(0.1)
crop = section1402.fetch(voi=voi, resolution_mm=-1)

In [ ]:
plotting.plot_img(crop, bg_img=None, cmap='gray')

In [ ]:
plotting.plot_img(crop, bg_img=bigbrain_template.fetch(), cmap='magma')